In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-09-09


Found rundate 2024-09-08 10:00:03 < 2024-09-09 at commit b83c07e0; returning commit 148ca6c


'2024-09-09 10:00:05'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-09-09 10:00:05, NJSP was reporting 446 YTD deaths
Current YTD Deaths (2025-09-09 10:00:01-04:00): 375
Previous year YTD Deaths (adjusted; 2024-09-09 10:00:05): 445.4497830089224
Projected 2025 total: 587.5671769161057
-15.8% change


61a25b2e94e02972242fd03ff0f35c1cfa24f47e: FAUQStats cache miss: 2024, Mon Sep 09 10:00:05 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,STREET,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,1,State Highway 1,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,129,State Highway 129,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,446,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,NaN,0.0,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,NaN,East Oak St,1.0,1.0,0.0,0.0,0.0,East Oak St,NaN,2024-01-05 14:12:00-05:00
13362,07,Essex,0701,Belleville Town,647,County 647 MP .76,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-05 18:08:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13417,11,Mercer,1103,Hamilton Twsp,33,State Highway 33 MP 2.84,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-09-06 21:19:00-04:00
13416,15,Ocean,1507,Toms River,37,State Highway 37,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-09-07 21:39:00-04:00
13419,19,Sussex,1914,Montague Twsp,650,County 650,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-09-08 10:29:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,16,3,6,0,24
Bergen,13,1,4,0,18
Burlington,16,5,4,0,23
Camden,15,2,9,1,26
Cape May,7,3,0,0,9
Cumberland,8,2,3,0,12
Essex,8,3,19,1,31
Gloucester,14,4,3,0,18
Hudson,5,1,4,1,10


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

68.9% through the year, 31.1% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,20,31,16
Bergen,driver,12,14,13
Burlington,driver,22,31,16
Camden,driver,13,19,15
Cape May,driver,3,8,7
...,...,...,...,...
Salem,crashes,11,14,11
Somerset,crashes,11,18,8
Sussex,crashes,10,12,5


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,41,0,25,5,11
Bergen,29,0,15,1,11
Burlington,35,0,23,5,9
Camden,36,1,22,3,12
Cape May,23,0,17,4,0
Cumberland,19,0,12,3,6
Essex,45,1,12,10,27
Gloucester,34,0,22,10,8
Hudson,20,1,9,1,10


In [9]:
projected.to_csv(PROJECTED_CSV)